In [1]:
%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
%env PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin

env: JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
env: PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin


In [2]:
import findspark
import pandas as pd
import numpy as np

import pyspark as psk
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

# import submitit

import time as t 
from datetime import date, datetime, timedelta
from joblib import Parallel, delayed
import multiprocessing
from functools import reduce  
from math import modf
import sqlite3
import os
import sys

from delta import * 
from delta.tables import *
from delta import configure_spark_with_delta_pip
# import covalent as ct

import subprocess
import re
import random

findspark.init()
findspark.find()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# os.environ['PYDEVD_DISABLE_FILE_VALIDATION']=1

spark = SparkSession.builder \
       .master("local[2]") \
       .appName("MyApp") \
       .config("spark.driver.maxResultSize", "20g")\
       .config("spark.driver.memory", "100g")\
       .getOrCreate()
sc = spark.sparkContext

# sqlContext = SQLContext(sc)


#both works
# 1: 
# spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")    #To resolve the error for p1075_38 to_timestamp formating: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '1/2/2019 20:40:00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
# Set Spark SQL legacy time parser policy to LEGACY to handle older date formats
# 2:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
# Increase the max fields in the string representation of a plan
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)  # Increase to 1000 or more as needed
spark.conf.set("spark.sql.execution.arrow.enabled", "false")

spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")

df_population = spark.sql("SELECT * FROM population")

ps: /storage/icds/RISE/sw8/anaconda/anaconda3/envs/tensorflow/lib/libuuid.so.1: no version information available (required by /usr/lib64/libblkid.so.1)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/01 01:25:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/01 01:25:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/01 01:25:46 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/04/01 01:25:47 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [3]:
spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data.csv") \
          .createOrReplaceTempView("VINs_data")

spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")

In [4]:
headerList = [    
          "calendar_day", 
          "f_1_dtc38_1th_15d", 
          "f_1_dtc38_2nd_15d", 
          "f_2_dtc38_1th_15d", 
          "f_2_dtc38_2nd_15d", 
          "f_3_dtc38_1th_15d", 
          "f_3_dtc38_2nd_15d", 
          "f_4_dtc38_1th_15d", 
          "f_4_dtc38_2nd_15d", 
          "f_5_dtc38_1th_15d", 
          "f_5_dtc38_2nd_15d", 
          "f_6_dtc38_1th_15d", 
          "f_6_dtc38_2nd_15d", 
          "f_7_dtc38_1th_15d", 
          "f_7_dtc38_2nd_15d", 
          "f_8_dtc38_1th_15d", 
          "f_8_dtc38_2nd_15d", 

          "f_1_dtc75_1th_15d", 
          "f_1_dtc75_2nd_15d", 
          "f_2_dtc75_1th_15d", 
          "f_2_dtc75_2nd_15d", 
          "f_3_dtc75_1th_15d", 
          "f_3_dtc75_2nd_15d", 
          "f_4_dtc75_1th_15d", 
          "f_4_dtc75_2nd_15d", 
          "f_5_dtc75_1th_15d", 
          "f_5_dtc75_2nd_15d", 
          "f_6_dtc75_1th_15d", 
          "f_6_dtc75_2nd_15d", 
          "f_7_dtc75_1th_15d", 
          "f_7_dtc75_2nd_15d", 
          "f_8_dtc75_1th_15d", 
          "f_8_dtc75_2nd_15d",


          "f_1_dtc77_1th_15d", 
          "f_1_dtc77_2nd_15d", 
          "f_2_dtc77_1th_15d", 
          "f_2_dtc77_2nd_15d", 
          "f_3_dtc77_1th_15d", 
          "f_3_dtc77_2nd_15d", 
          "f_4_dtc77_1th_15d", 
          "f_4_dtc77_2nd_15d", 
          "f_5_dtc77_1th_15d", 
          "f_5_dtc77_2nd_15d", 
          "f_6_dtc77_1th_15d", 
          "f_6_dtc77_2nd_15d", 
          "f_7_dtc77_1th_15d", 
          "f_7_dtc77_2nd_15d", 
          "f_8_dtc77_1th_15d", 
          "f_8_dtc77_2nd_15d",


          "f_1_dtc86_1th_15d", 
          "f_1_dtc86_2nd_15d", 
          "f_2_dtc86_1th_15d", 
          "f_2_dtc86_2nd_15d", 
          "f_3_dtc86_1th_15d", 
          "f_3_dtc86_2nd_15d", 
          "f_4_dtc86_1th_15d", 
          "f_4_dtc86_2nd_15d", 
          "f_5_dtc86_1th_15d", 
          "f_5_dtc86_2nd_15d", 
          "f_6_dtc86_1th_15d", 
          "f_6_dtc86_2nd_15d", 
          "f_7_dtc86_1th_15d", 
          "f_7_dtc86_2nd_15d", 
          "f_8_dtc86_1th_15d", 
          "f_8_dtc86_2nd_15d",


          "f_1_dtc92_1th_15d", 
          "f_1_dtc92_2nd_15d", 
          "f_2_dtc92_1th_15d", 
          "f_2_dtc92_2nd_15d", 
          "f_3_dtc92_1th_15d", 
          "f_3_dtc92_2nd_15d", 
          "f_4_dtc92_1th_15d", 
          "f_4_dtc92_2nd_15d", 
          "f_5_dtc92_1th_15d", 
          "f_5_dtc92_2nd_15d", 
          "f_6_dtc92_1th_15d", 
          "f_6_dtc92_2nd_15d", 
          "f_7_dtc92_1th_15d", 
          "f_7_dtc92_2nd_15d", 
          "f_8_dtc92_1th_15d", 
          "f_8_dtc92_2nd_15d",


          "f_1_dtc94_1th_15d", 
          "f_1_dtc94_2nd_15d", 
          "f_2_dtc94_1th_15d", 
          "f_2_dtc94_2nd_15d", 
          "f_3_dtc94_1th_15d", 
          "f_3_dtc94_2nd_15d", 
          "f_4_dtc94_1th_15d", 
          "f_4_dtc94_2nd_15d", 
          "f_5_dtc94_1th_15d", 
          "f_5_dtc94_2nd_15d", 
          "f_6_dtc94_1th_15d", 
          "f_6_dtc94_2nd_15d", 
          "f_7_dtc94_1th_15d", 
          "f_7_dtc94_2nd_15d", 
          "f_8_dtc94_1th_15d", 
          "f_8_dtc94_2nd_15d",


          "f_1_dtc0401_1th_15d", 
          "f_1_dtc0401_2nd_15d", 
          "f_2_dtc0401_1th_15d", 
          "f_2_dtc0401_2nd_15d", 
          "f_3_dtc0401_1th_15d", 
          "f_3_dtc0401_2nd_15d", 
          "f_4_dtc0401_1th_15d", 
          "f_4_dtc0401_2nd_15d", 
          "f_5_dtc0401_1th_15d", 
          "f_5_dtc0401_2nd_15d", 
          "f_6_dtc0401_1th_15d", 
          "f_6_dtc0401_2nd_15d", 
          "f_7_dtc0401_1th_15d", 
          "f_7_dtc0401_2nd_15d", 
          "f_8_dtc0401_1th_15d", 
          "f_8_dtc0401_2nd_15d",


          "f_1_dtc2457_1th_15d", 
          "f_1_dtc2457_2nd_15d", 
          "f_2_dtc2457_1th_15d", 
          "f_2_dtc2457_2nd_15d", 
          "f_3_dtc2457_1th_15d", 
          "f_3_dtc2457_2nd_15d", 
          "f_4_dtc2457_1th_15d", 
          "f_4_dtc2457_2nd_15d", 
          "f_5_dtc2457_1th_15d", 
          "f_5_dtc2457_2nd_15d", 
          "f_6_dtc2457_1th_15d", 
          "f_6_dtc2457_2nd_15d", 
          "f_7_dtc2457_1th_15d", 
          "f_7_dtc2457_2nd_15d", 
          "f_8_dtc2457_1th_15d", 
          "f_8_dtc2457_2nd_15d",

          "if_parts_replaced_in_1th_15d", 
          "if_parts_replaced_in_2nd_15d"]

In [5]:
def is_valid_vin(vin: str) -> int:
    """
    Returns 1 if the VIN is valid, 0 if not.
    A valid VIN is exactly 17 characters long and does not contain I, O, or Q.
    """
    vin = vin.strip().upper()

    # Rule 1: VIN must be exactly 17 characters
    if len(vin) != 17:
        return 0

    # Rule 2: Must only contain A-H, J-N, P, R-Z, and 0-9 (excluding I, O, Q)
    if not re.match("^[A-HJ-NPR-Z0-9]{17}$", vin):
        return 0

    return 1

In [6]:
# Input paths
VINs_data_file_path = '/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data.csv'
resulted_data_path = '/storage/home/yqf5148/work/volvoPennState/data/dataset/resultedData.csv'

# Columns: Keep selected + headers
selected_features_from_population = ['VIN', 'ENGINE_SIZE', 'ENGINE_HP', 'VEH_TYPE'] + [s for s in df_population.columns if 'KOLA' in s]
all_columns_names = selected_features_from_population + headerList

# 🔹 Step 1: Load VINs_data.csv
df_vins = pd.read_csv(VINs_data_file_path, names=["VIN", "TOTAL_ROWS"], dtype={"VIN": str, "TOTAL_ROWS": int}, header=None)

# 🔹 Step 2: Load resultedData.csv (skip bad lines)
df_resulted = pd.read_csv(resulted_data_path, names=all_columns_names, header=None, on_bad_lines='skip')

# 🔹 Step 3: Convert calendar_day to numeric
df_resulted["calendar_day"] = pd.to_numeric(df_resulted["calendar_day"], errors='coerce')

# 🔹 Step 4: Group by VIN → max calendar_day
grouped = df_resulted.groupby("VIN")["calendar_day"].max().reset_index()
grouped.columns = ["VIN", "MAX_CALENDAR_DAY"]

# 🔹 Step 5: Build dictionary of existing VINs
vins_dict = dict(zip(df_vins["VIN"], df_vins["TOTAL_ROWS"]))

# 🔹 Step 6: Update or add VINs
updated_rows = []

for _, row in grouped.iterrows():
    vin = row["VIN"]
    max_day_val = row["MAX_CALENDAR_DAY"]

    if(is_valid_vin(vin)==0):
        print(f" Skipping VIN: {vin} because it is invalid.")
        continue
        
    if pd.isna(max_day_val):
        print(f" Skipping VIN: {vin} → MAX_CALENDAR_DAY is NaN")
        continue

    max_day = int(max_day_val)

    if vin in vins_dict:
        print(f" Updating VIN: {vin} → TOTAL_ROWS: {vins_dict[vin]} → {max_day}")
    else:
        print(f" Adding new VIN: {vin} → TOTAL_ROWS: {max_day}")

    updated_rows.append([vin, max_day])

# 🔹 Step 7: Add VINs from original not in grouped
existing_vins_set = set(grouped["VIN"])
for vin, total_rows in vins_dict.items():
    if vin not in existing_vins_set:
        updated_rows.append([vin, total_rows])
        print(f" Keeping existing VIN: {vin} → TOTAL_ROWS: {total_rows}")

# 🔹 Step 8: Save updated VINs
df_final = pd.DataFrame(updated_rows, columns=["VIN", "TOTAL_ROWS"])
df_final.to_csv(VINs_data_file_path, index=False, header=False)

print(f"\n VINs_data.csv successfully updated. Total VINs: {len(df_final)}")


/tmp/ipykernel_1062966/3116708125.py:13: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_resulted = pd.read_csv(resulted_data_path, names=all_columns_names, header=None, on_bad_lines='skip')


 Skipping VIN: -AGS because it is invalid.
 Skipping VIN: -ST because it is invalid.
 Skipping VIN: .0 because it is invalid.
 Skipping VIN: 0 because it is invalid.
 Skipping VIN: 0.0 because it is invalid.
 Skipping VIN: 1 because it is invalid.
 Adding new VIN: 1M1AN07Y3GM021042 → TOTAL_ROWS: 414
 Adding new VIN: 1M1AN2AY5LM001812 → TOTAL_ROWS: 2084
 Adding new VIN: 1M1AN2GY3KM006268 → TOTAL_ROWS: 29
 Adding new VIN: 1M1AN3GY4LM019358 → TOTAL_ROWS: 29
 Adding new VIN: 1M1AN4GY0MM022958 → TOTAL_ROWS: 1914
 Adding new VIN: 1M1AN4GY0MM022961 → TOTAL_ROWS: 2339
 Adding new VIN: 1M1AN4GY7KM008343 → TOTAL_ROWS: 779
 Adding new VIN: 1M1AN4GY8MM021279 → TOTAL_ROWS: 2199
 Adding new VIN: 1M1AN4GYXLM018348 → TOTAL_ROWS: 29
 Adding new VIN: 1M1AW01Y2FM006911 → TOTAL_ROWS: 1433
 Adding new VIN: 1M1AW01Y5JM010413 → TOTAL_ROWS: 24
 Adding new VIN: 1M1AW02Y1FM050302 → TOTAL_ROWS: 29
 Adding new VIN: 1M1AW02Y7FM051728 → TOTAL_ROWS: 764
 Adding new VIN: 1M1AW07Y4FM045037 → TOTAL_ROWS: 68
 Adding new